#Introdução à inteligência artificial

## Aula 4 - Como treinar uma rede neural artificial

A rede que construímos na aula anterior não é tão inteligente, ela não sabe nada sobre nossos dígitos manuscritos. As redes neurais com ativações não lineares funcionam como aproximadores de funções universais. Há alguma função que mapeia sua entrada para a saída. Por exemplo, imagens de dígitos manuscritos para probabilidades de classe. O poder das redes neurais é que podemos treiná-las para aproximar essa função e basicamente qualquer função que tenha dados e tempo de computação suficientes.

A princípio, a rede é ingênua, não conhece a função que mapeia as entradas para as saídas. Nós treinamos a rede, mostrando exemplos de dados reais e, em seguida, ajustando os parâmetros da rede para que ela se aproxime dessa função.

Para encontrar esses parâmetros, precisamos saber quão mal a rede está prevendo as saídas reais. Para isso, calculamos uma **função de perda** (também chamada de custo), uma medida do nosso erro de previsão. Por exemplo, a perda quadrática média é frequentemente usada em problemas de regressão e classificação binária

$$
\large C = \frac{1}{2n}\sum_i^n{\left(y_i - \hat{y}_i\right)^2}
$$

onde $ n $ é o número de exemplos de treinamento, $ y_i $ são os rótulos verdadeiros e $ \hat {y}_i $ são os rótulos previstos.

Ao minimizar essa perda com relação aos parâmetros de rede, podemos encontrar configurações em que a perda é mínima e a rede é capaz de prever as etiquetas corretas com alta precisão. Achamos esse mínimo usando um processo chamado **descida de gradiente**. O gradiente é a inclinação da função de perda e aponta na direção da mudança mais rápida. Para chegar ao mínimo na menor quantidade de tempo, queremos seguir o gradiente (para baixo). Você pode pensar nisso como descer uma montanha seguindo a ladeira mais íngreme até a base.

<div align=center>
<img src='https://blog.paperspace.com/content/images/2018/05/fastlr.png' width=450px>
</div>


### Retropropagação


Para redes de camada única, a descida em gradiente é simples de implementar. No entanto, é mais complicado para redes neurais mais profundas e multicamadas como a que construímos. Complicado o suficiente para levar cerca de 30 anos até os pesquisadores descobrirem como treinar redes multicamadas.

O treinamento de redes multicamadas é feito por meio de **retropropagação**, que é realmente apenas uma aplicação da regra da cadeia de cálculo. É mais fácil entender se convertermos uma rede de duas camadas em uma representação gráfica.

<div align=center>
<img src='https://miro.medium.com/max/1485/0*ZlMNumCS7Z1wQgax.' width=450px>

Figura 1 - Passagem direta
</div>


<div align=center>
<img src='https://miro.medium.com/max/1413/0*U7Zg0fqsv4dN-TFO.' width=450px>

Figura 2 - Passagem inversa em passos
</div>

<div align=center>
<img src='https://miro.medium.com/max/1413/0*VHvavv03ptQ4jBiP.' width=450px>

Figura 3 - Passagem inversa
</div>


Na passagem direta pela rede, nossos dados e operações vão de baixo para cima aqui. Passamos a entrada $ x $ por uma transformação linear $ L_1 $ com pesos $ W_1 $ e viés $ b_1 $. A saída passa pela operação sigmóide $ S $ e outra transformação linear $ L_2 $. Finalmente, calculamos a perda $ C $. Usamos a perda como uma medida de quão ruins são as previsões da rede. O objetivo, então, é ajustar os pesos e desvios para minimizar a perda.

Para treinar os pesos com descida de gradiente, propagamos o gradiente da perda para trás através da rede. Cada operação possui algum gradiente entre as entradas e saídas. À medida que enviamos os gradientes para trás, multiplicamos o gradiente de entrada pelo gradiente da operação. Matematicamente, isso é realmente apenas o cálculo do gradiente da perda com relação aos pesos usando a regra da cadeia.

$$
\large \frac{\partial C}{\partial W_1} = \frac{\partial L_1}{\partial W_1} \frac{\partial S}{\partial L_1} \frac{\partial L_2}{\partial S} \frac{\partial C}{\partial L_2}
$$

Atualizamos nossos pesos usando esse gradiente com alguma taxa de aprendizado $ \alpha $.

$$
\large W^\prime_1 = W_1 - \alpha \frac{\partial C}{\partial W_1}
$$

A taxa de aprendizado $ \alpha $ é configurada de forma que as etapas de atualização de peso sejam pequenas o suficiente para que o método iterativo seja estabelecido no mínimo.

### Perdas no PyTorch

Vamos começar vendo como calculamos a perda com o PyTorch. Através do módulo `nn`, o PyTorch fornece perdas como a perda de entropia cruzada (` nn.CrossEntropyLoss`). Você geralmente verá a perda atribuída ao 'critério'. Conforme observado na última parte, com um problema de classificação como MNIST, estamos usando a função softmax para prever probabilidades de classe. Com uma saída softmax, você deseja usar a entropia cruzada como perda. Para realmente calcular a perda, primeiro defina o critério e depois passe a saída da sua rede e os rótulos corretos.

Algo realmente importante a ser observado aqui. Examinando [a documentação para `nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss),

> Este critério combina `nn.LogSoftmax()` e `nn.NLLLoss()` em uma única classe.
>
> A entrada deve conter pontuações para cada classe.

Isso significa que precisamos passar a produção bruta de nossa rede para a perda, não a produção da função softmax. Essa saída bruta geralmente é chamada de *logits* ou *scores*. Usamos os logits porque o softmax fornece probabilidades que geralmente ficam muito próximas de zero ou um, mas os números de ponto flutuante não podem representar com precisão valores próximos de zero ou um ([leia mais aqui](https://docs.python.org/3/tutorial/floatingpoint.html)). Geralmente, é melhor evitar cálculos com probabilidades, normalmente usamos probabilidades de log.

In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms


# Defina uma transformação para normalizar os dados
transform = transforms.Compose([
transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

# Faça o download e carregue os dados de treinamento
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [0]:
# Construa uma rede neural
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10))

# Defina a perda
criterion = nn.CrossEntropyLoss()

# Pega a informação
images, labels = next(iter(trainloader))
# "Achata" a imagem
images = images.view(images.shape[0], -1)

# Passe para frente, obtenha nossos logits
logits = model(images)
# Calcular a perda com os logits e os rótulos
loss = criterion(logits, labels)

print(loss)

É mais conveniente criar o modelo com uma saída log-softmax usando `nn.LogSoftmax` ([documentação](https://pytorch.org/docs/stable/nn.html#torch.nn.LogSoftmax)). Então você pode obter as probabilidades reais pegando o exponencial `torch.exp (output)`. Com uma saída log-softmax, você deseja usar a perda de probabilidade de log negativa, `nn.NLLLoss` ([documentação](https://pytorch.org/docs/stable/nn.html#torch.nn.NLLLoss)) .

> **Exercício:** Construa um modelo que retorne o log-softmax como saída (lembrando que o parâmetro dim=1) e calcule a perda usando a perda de probabilidade de log negativa.

In [0]:
# Construa uma rede neural 
model = None

# Defina a perda
criterion = None

# Pega a informação
images, labels = next(iter(trainloader))
# "Achata" a imagem
images = images.view(images.shape[0], -1)

# Passe para frente, obtenha as probabilidades log
logps = model(images)
# Calcular a perda com as probabilidades log e os rótulos
loss = criterion(logps, labels)

print(loss)

### Autograd

Agora que sabemos como calcular uma perda, como a usamos para realizar a retropropagação? O PyTorch fornece um módulo, `autograd`, para calcular automaticamente os gradientes dos tensores. Podemos usá-lo para calcular os gradientes de todos os nossos parâmetros em relação à perda. O Autograd trabalha mantendo o controle das operações realizadas nos tensores e depois retrocedendo nessas operações, calculando gradientes ao longo do caminho. Para garantir que o PyTorch acompanhe as operações em um tensor e calcule os gradientes, você precisa definir `require_grad = True` em um tensor. Você pode fazer isso na criação com a palavra-chave `require_grad` ou a qualquer momento com` x.requires_grad_(True) `.

Você pode desativar gradientes para um bloco de código com o conteúdo `torch.no_grad()`:
```python
x = torch.zeros(1, requires_grad=True)
>>> with torch.no_grad():
...     y = x * 2
>>> y.requires_grad
False
```

Além disso, você pode ativar ou desativar gradientes completamente com `torch.set_grad_enabled(True|False)`.

Os gradientes são calculados com relação a alguma variável `z` com` z.backward() `. Isso faz um retrocesso nas operações que criaram `z`.

In [0]:
x = torch.randn(2,2, requires_grad=True)
print(x)

In [0]:
y = x**2
print(y)

Abaixo podemos ver a operação que criou `y`, uma operação de potência `PowBackward0`.

In [0]:
## grad_fn mostra a função que gerou essa variável
print(y.grad_fn)

O módulo autograd monitora essas operações e sabe como calcular o gradiente para cada uma. Dessa forma, é capaz de calcular os gradientes para uma cadeia de operações, com relação a qualquer tensor. Vamos reduzir o tensor `y` para um valor escalar, a média.

In [0]:
z = y.mean()
print(z)

Você pode verificar os gradientes para `x` e `y`, mas eles estão vazios no momento.

In [0]:
print(x.grad)

Para calcular os gradientes, você precisa executar o método `.backward` em uma variável, `z` por exemplo. Isso calculará o gradiente para `z` com relação a `x`

$$
\frac{\partial z}{\partial x} = \frac{\partial}{\partial x}\left[\frac{1}{n}\sum_i^n x_i^2\right] = \frac{x}{2}
$$

In [0]:
z.backward()
print(x.grad)
print(x/2)

Esses cálculos de gradientes são particularmente úteis para redes neurais. Para o treinamento, precisamos dos gradientes dos pesos em relação ao custo. Com o PyTorch, executamos os dados adiante pela rede para calcular a perda e depois retrocedemos para calcular os gradientes em relação à perda. Quando tivermos os gradientes, podemos fazer um passo de descida de gradiente.

### Perda e Autograd juntos

Quando criamos uma rede com o PyTorch, todos os parâmetros são inicializados com `require_grad = True`. Isso significa que, quando calculamos a perda e chamamos `loss.backward()`, os gradientes para os parâmetros são calculados. Esses gradientes são usados para atualizar os pesos com a descida do gradiente. Abaixo, você pode ver um exemplo de cálculo dos gradientes usando um passe para trás.

In [0]:
# Criando a rede neural e definindo o critério
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images)
loss = criterion(logps, labels)

In [0]:
print('Antes do passe inverso: \n', model[0].weight.grad)

loss.backward()

print('Depois do passe inverso: \n', model[0].weight.grad)

### Treinando a rede!

Há uma última peça que precisamos para começar o treinamento, um otimizador que usaremos para atualizar os pesos com os gradientes. Nós obtemos isso do pacote PyTorch [`optim`](https://pytorch.org/docs/stable/optim.html). Por exemplo, podemos usar a descida de gradiente estocástico com `optim.SGD`. Você pode ver como definir um otimizador abaixo.

In [0]:
from torch import optim

# Os otimizadores exigem os parâmetros para otimizar e uma taxa de aprendizado
optimizer = optim.SGD(model.parameters(), lr=0.01)

Agora sabemos como usar todas as partes individuais, então é hora de ver como elas funcionam juntas. Vamos considerar apenas uma etapa de aprendizado antes de percorrer todos os dados. O processo geral com o PyTorch:

* Faça um passe para frente através da rede
* Use a saída de rede para calcular a perda
* Execute um retrocesso na rede com `loss.backward()` para calcular os gradientes
* Dê um passo com o otimizador para atualizar os pesos

Abaixo, vou seguir uma etapa do treinamento e imprimir os pesos e gradientes para que você possa ver como isso muda. Note que eu tenho uma linha de código `optimizer.zero_grad()`. Quando você faz várias passagens para trás com os mesmos parâmetros, os gradientes são acumulados. Isso significa que você precisa zerar os gradientes em cada passe de treinamento ou reter gradientes dos lotes de treinamento anteriores.

In [0]:
print('Pesos iniciais - ', model[0].weight)

images, labels = next(iter(trainloader))
images.resize_(64, 784)

# Limpe os gradientes, faça isso porque os gradientes são acumulados
optimizer.zero_grad()

# Passe para frente, depois para trás e atualize pesos
output = model(images)
loss = criterion(output, labels)
loss.backward()
print('Gradiente -', model[0].weight.grad)

In [0]:
# Dê um passo de atualização e atualiza os novos pesos
optimizer.step()
print('Pesos atualizados - ', model[0].weight)

### Treinamento de verdade

Agora vamos colocar esse algoritmo em um loop para que possamos percorrer todas as imagens. Em alguma nomenclatura, uma passagem por todo o conjunto de dados é chamada de *época*. Então, aqui vamos percorrer o `trainloader 'para obter nossos lotes de treinamento. Para cada lote, faremos um passe de treinamento em que calculamos a perda, passamos para trás e atualizamos os pesos.

> **Exercício:** Implemente o passe de treinamento para nossa rede. Se você o implementou corretamente, a perda de treinamento diminui a cada época.

In [0]:
model = None

criterion = None
optimizer = None

epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
    
        # TODO: Passe de treino
        # Limpe o gradiente do otimizador
        # Faça um passe pra frente usando o modelo e as imagens, obtendo as probabilidades log
        # Use o critério com as probabilidades e os 'labels, obtendo a perda "loss"
        # Realize um "passe pra trás" na perda
        # Dê um passo no otimizador
                
        running_loss += loss.item()
    else:
        print(f"Perda de treinamento: {running_loss/len(trainloader)}")

Com a rede treinada, podemos verificar suas previsões.

In [0]:
# Função auxiliar
def view_classify(img, ps, version="MNIST"):
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Probabilidade de classe')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


images, labels = next(iter(trainloader))

img = images[1].view(1, 784)
# Desative gradientes para acelerar esta parte
with torch.no_grad():
    logps = model(img)

# Os resultados da rede são probabilidades de log, precisam ser exponenciais para probabilidades
ps = torch.exp(logps)
view_classify(img.view(1, 28, 28), ps)